In [29]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))

        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        predicted_class = torch.where(out > 0.5, torch.tensor(0.0), torch.tensor(1.0))

        return predicted_class

input_size = 2
hidden_size = 32
num_layers = 10
output_size = 1

model = LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)


In [33]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

data_dir = '../data/emg_rms/'

class CustomDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(data_dir, self.file_list[idx])
        df = pd.read_csv(file_path)

        features = df[['V1', 'V2']].values
        label = df[df.columns[-1]].loc[0]

        return torch.FloatTensor(features), label


In [ ]:
file_list = os.listdir(data_dir)

train_files, val_files = train_test_split(file_list, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_files)
val_dataset = CustomDataset(val_files)

batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs[0], labels)
        print(loss)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        total_correct = 0
        total_samples = 0

        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs[0], labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        average_val_loss = val_loss / len(val_loader)
        accuracy = total_correct / total_samples

        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy:.4f}')

In [41]:
import numpy as np

data_dir = '../data/emg_rms/'
transformed_dfs = []

def calculate_wave_shape(x):
    x = np.array(x, dtype=np.float32)
    differences = np.abs(x[1:] - x[:-1])

    wave_shape = differences.sum()

    return wave_shape

def calculate_mean_signal_energy(x):
    # Convert the input to numpy array
    x = np.array(x)

    # Calculate the sum of squared values
    sum_squared_values = np.sum(x**2)

    # Calculate the Mean Signal Energy
    mean_signal_energy = sum_squared_values / len(x)

    return mean_signal_energy

# Iterate through each file in the dataset directory
for filename in os.listdir(data_dir):
    file_path = os.path.join(data_dir, filename)

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Calculate MAE and MAV for each channel
    mae_v1 = df['V1'].abs().mean()
    ws_v1 = calculate_wave_shape(df['V1'])
    sen_v1 = calculate_mean_signal_energy(df['V1'])

    mae_v2 = df['V2'].abs().mean()
    ws_v2 = calculate_wave_shape(df['V2'])
    sen_v2 = calculate_mean_signal_energy(df['V2'])
    # Get the label (assuming it's in the first row)
    label = df[df.columns[-1]].iloc[0]

    # Create a new DataFrame with the calculated features and label
    features = {
        'MAE_V1': mae_v1,
        'WS_V1': ws_v1,
        'SEN_V1': sen_v1,
        'MAE_V2': mae_v2,
        'WS_V2': ws_v2,
        'SEN_V2': sen_v2,
        'Label': label
    }

    transformed_df = pd.DataFrame([features])

    # Append the DataFrame to the list
    transformed_dfs.append(transformed_df)

# Concatenate all DataFrames into one
final_df = pd.concat(transformed_dfs, ignore_index=True)

# Display the resulting DataFrame
print(final_df)

        MAE_V1   WS_V1         SEN_V1      MAE_V2   WS_V2         SEN_V2  \
0   348.522222  2644.0  125437.677778  379.955556   967.0  145912.822222   
1   354.861386  2650.0  129046.584158  385.287129  1103.0  150835.900990   
2   439.800000  1670.0  208374.200000  339.250000   702.0  115821.116667   
3   355.010000  2559.0  129251.330000  372.510000  1051.0  141210.510000   
4   361.648649  1531.0  136412.567568  368.837838   317.0  138069.594595   
5   335.500000  2376.0  120302.933333  363.650000   643.0  132562.516667   
6   340.886364  2301.0  126652.204545  360.318182   446.0  130171.045455   
7   437.260000  1335.0  204210.060000  335.060000   951.0  113620.260000   
8   342.957143  2582.0  123893.157143  374.057143   733.0  140688.142857   
9   358.355263  2849.0  134479.697368  377.736842   801.0  144415.605263   
10  343.000000  2648.0  122369.945946  383.162162   899.0  147863.972973   
11  399.200000  2667.0  179185.828571  364.085714  1918.0  144233.971429   
12  372.4800